# ft_printf

## Makefile

In [159]:
%%file ../ft_printf/Makefile
# **************************************************************************** #
#                                                                              #
#                                                         :::      ::::::::    #
#    Makefile                                           :+:      :+:    :+:    #
#                                                     +:+ +:+         +:+      #
#    By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+         #
#                                                 +#+#+#+#+#+   +#+            #
#    Created: 2022/12/29 22:12:48 by dpentlan          #+#    #+#              #
#    Updated: 2022/12/29 22:12:50 by dpentlan         ###   ########.fr        #
#                                                                              #
# **************************************************************************** #

NAME = libftprintf.a

SRCS = ft_printf.c \
    ft_printf_utils_1.c \
    ft_printf_utils_2.c

OBJS = ${SRCS:.c=.o}

INCS = ft_printf.h

CFLAGS = -Wall -Wextra -Werror

CC = cc

SLIB = ar rcs

all: $(NAME)

$(NAME): $(OBJS) $(INCS)
	$(SLIB) $(NAME) -o $(OBJS)

%.o: %.c
	$(CC) $(CFLAGS) -c $< -o $(<:.c=.o)

clean:
	rm -f $(OBJS)

fclean:	clean
	rm -f $(NAME)

re:	fclean all


Overwriting ../ft_printf/Makefile


## ft_printf.h

In [220]:
%%file ../ft_printf/ft_printf.h
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   ft_printf.h                                        :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2022/12/30 18:32:59 by dpentlan          #+#    #+#             */
/*   Updated: 2022/12/30 18:33:01 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#ifndef FT_PRINTF_H
# define FT_PRINTF_H
# include <unistd.h>
# include <stdarg.h>
# include <stddef.h>

int ft_printf(const char *, ...);
void	ft_putchar_fd(char c, int fd, int *print_count);
void	ft_putstr_fd(char *s, int fd, int *print_count);
void	ft_putnbr_fd(int n, int fd, int *print_count);
void	ft_putpercent(int *print_count);
void	ft_dec_to_hex_lower(unsigned int n, int fd, int *print_count);
void	ft_dec_to_hex_upper(unsigned int n, int fd, int *print_count);
void	ft_p_to_hex(void *p, int fd, int *print_count);
void	ft_putunbr_fd(unsigned int n, int fd, int *print_count);

#endif

Overwriting ../ft_printf/ft_printf.h


## ft_printf.c

In [282]:
%%file ../ft_printf/ft_printf.c
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   ft_printf.c                                        :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2022/12/31 14:22:48 by dpentlan          #+#    #+#             */
/*   Updated: 2022/12/31 14:22:49 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#include "ft_printf.h"

/*	*** ft_pointer_handle (42 pointer handler) ***
 *
 *	Additional function to check NULL status of pointer.
 *	If NULL, prints "(nil)" and returns.
 *	If not NULL, prints the address.
 */

void	ft_pointer_handle(va_list parg, int fd, int *print_count)
{
	void	*ptr;

	ptr = va_arg(parg, void *);
	if (!ptr)
	{
		write(fd, "(nil)", 5);
		*print_count += 5;
	}
	else
	{
		write (fd, "0x", 2);
		*print_count += 2;
		ft_p_to_hex(ptr, fd, print_count);
	}
}


/*  *** format_switch (format switch file descriptor) ***
 *
 *  Acts as a switch statement that determines which function to call
 *  Based on the format specifier used in ft_printf.
 *	Takes the character after a % from input string s in function ft_printf
 *	Takes va_list pointer 'parg'.
 *	Runs va_arg macro to get the next item in va_list. At this point we know
 *	the type thanks to the switch statement.
 *	Also takes a file descriptor.
 *	Returns integer 1 on success.
 */

static int	format_switch_fd(char c, va_list parg, int fd, int *print_count)
{
	if (c == 'c')
		ft_putchar_fd(va_arg(parg, int), fd, print_count);
	else if (c == 's')
		ft_putstr_fd(va_arg(parg, char *), fd, print_count);
	else if (c == 'p')
		ft_pointer_handle(parg, fd, print_count);
	else if (c == 'd')
		ft_putnbr_fd(va_arg(parg, int), fd, print_count);
	else if (c == 'i')
		ft_putnbr_fd(va_arg(parg, int), fd, print_count);
	else if (c == 'u')
		ft_putunbr_fd((unsigned int)va_arg(parg, int), fd, print_count);
	else if (c == 'x')
		ft_dec_to_hex_lower(va_arg(parg, int), fd, print_count);
	else if (c == 'X')
		ft_dec_to_hex_upper(va_arg(parg, int), fd, print_count);
	else if (c == '%')
		ft_putpercent(print_count);
	return (1);
}

/*  *** ft_printf (42 print format) ***
 *
 *  Takes a string followed by a variable number of arguments corresponding
 *  to format specifiers in the string.
 *	Uses a loop to iterate over string. When a % is found, activates a switch
 *	statement function and passes pointer to va_list for further execution.
 *	Returns integer number of character processed from string 's'.
 */

int ft_printf(const char *s, ...)
{
	size_t	i;
	va_list	parg;
	int		fd;
	int		print_count;

	i = 0;
	fd = 1;
	print_count = 0;
	va_start(parg, s);
	while (s[i])
	{
		if (s[i] == '%')
		{
			format_switch_fd(s[i + 1], parg, fd, &print_count);
			i++;
		}
		else
		{
			write(fd, &s[i], 1);
			print_count++;
		}
		i++;
	}
	va_end(parg);
	return (print_count);
}

Overwriting ../ft_printf/ft_printf.c


## ft_printf_utils_1.c

In [222]:
%%file ../ft_printf/ft_printf_utils_1.c
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   ft_printf_utils_1.c                                :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2022/12/31 14:41:16 by dpentlan          #+#    #+#             */
/*   Updated: 2022/12/31 14:41:19 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#include "ft_printf.h"

/*  *** ft_putchar_fd (42 put character file descriptor) ***
 *
 *  Outputs the character 'c' to file descriptor.
 *  Returns nothing.
 */

void	ft_putchar_fd(char c, int fd, int *print_count)
{
	write(fd, &c, 1);
	(*print_count)++;
}

/*  *** ft_strlen (42 string length) ***
 *	*** from libft ***
 *
 *  Takes a character pointer.
 *  Returns size_t number of 8 bit memory spaces between input pointer and
 *  the first null terminator.
 */

size_t	ft_strlen(const char *str)
{
	size_t	i;

	i = 0;
	while (str && str[i])
		i++;
	return (i);
}

/*  *** ft_putstr_fd (42 put string file descriptor) ***
 *
 *  Outputs the string 's' to the given file descriptor.
 *  Returns nothing.
 */

void	ft_putstr_fd(char *s, int fd, int *print_count)
{
	size_t	str_len;

	str_len = 0;
	//if (s == 0)
		//return ;
	str_len = ft_strlen(s);
	write(fd, s, str_len);
	(*print_count) += str_len;
}

/*  *** ft_putnbr_fd (42 put number file descriptor) ***
 *
 *  Outputs the integer 'n' to the given file descriptor.
 *  Returns nothing.
 */

void	ft_putnbr_fd(int n, int fd, int *print_count)
{
	long	nbl;
	char	c;

	nbl = (long)n;
	if (nbl < 0)
	{
		write(fd, "-", 1);
		(*print_count)++;
	}
	if (nbl / 10 != 0)
	{
		if (nbl < 0)
			ft_putnbr_fd((nbl / 10) * -1, fd, print_count);
		else
			ft_putnbr_fd(nbl / 10, fd, print_count);
	}
	if (nbl < 0)
		c = (((nbl % 10) * -1) + '0');
	else
		c = (nbl % 10 + '0');
	write(fd, &c, 1);
	(*print_count)++;
	return ;
}

/*	*** ft_putpercent (42 put percent)
 *
 *	Writes a percent to stdout.
 *	Returns nothing.
 */

 void	ft_putpercent(int *print_count)
{
	write(1, "%", 1);
	(*print_count)++;
}

Overwriting ../ft_printf/ft_printf_utils_1.c


## ft_printf_utils_2.c

In [223]:
%%file ../ft_printf/ft_printf_utils_2.c
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   ft_printf_utils_2.c                                :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2022/12/31 14:41:27 by dpentlan          #+#    #+#             */
/*   Updated: 2022/12/31 14:41:28 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#include "ft_printf.h"

/*	*** ft_dec_to_hex_upper (42 decimal to hexidecimal uppercase) ***
 *
 *	Takes an integer 'n' and writes its hexadecimal representation to
 *	file descriptor.
 */

void	ft_dec_to_hex_upper(unsigned int n, int fd, int *print_count)
{
	long	nbl;
	char	c;

	nbl = (long)n;
	if (nbl < 0)
	{
		write(fd, "-", 1);
		(*print_count)++;
	}
	if (nbl / 16 != 0)
	{
		if (nbl < 0)
			ft_dec_to_hex_upper((nbl / 16) * -1, fd, print_count);
		else
			ft_dec_to_hex_upper(nbl / 16, fd, print_count);
	}
	if (nbl < 0)
		if (nbl % 16 > 9)
			c = (((nbl % 16) * -1) + 55);
		else
			c = (((nbl % 16) * -1) + '0');
	else
		if (nbl % 16 > 9)
			c = ((nbl % 16) + 55);
		else
			c = ((nbl % 16) + '0');
	write(fd, &c, 1);
	(*print_count)++;
	return ;
}

/*	*** ft_dec_to_hex_lower (42 decimal to hexidecimal lowercase) ***
 *
 *	Takes an integer 'n' and writes its hexadecimal representation to
 *	file descriptor.
 */

void	ft_dec_to_hex_lower(unsigned int n, int fd, int *print_count)
{
	long	nbl;
	char	c;

	nbl = (long)n;
	if (nbl < 0)
	{
		write(fd, "-", 1);
		(*print_count)++;
	}
	if (nbl / 16 != 0)
	{
		if (nbl < 0)
			ft_dec_to_hex_lower((nbl / 16) * -1, fd, print_count);
		else
			ft_dec_to_hex_lower(nbl / 16, fd, print_count);
	}
	if (nbl < 0)
		if (nbl % 16 > 9)
			c = (((nbl % 16) * -1) + 87);
		else
			c = (((nbl % 16) * -1) + '0');
	else
		if (nbl % 16 > 9)
			c = ((nbl % 16) + 87);
		else
			c = ((nbl % 16) + '0');
	write(fd, &c, 1);
	(*print_count)++;
	return ;
}

/*	*** ft_p_to_hex (42 pointer to hexidecimal) ***
 *
 *	Takes a void pointer, casts it to an unsigned long long and
 *	recursively calls ft_p_to_hex to print address as hexidecimal to
 *	file descriptor.
 *	Returns nothing.
 */

void	ft_p_to_hex(void *p, int fd, int *print_count)
{
	unsigned long long	nbl;
	char	c;

	nbl = (unsigned long long)p;
	if (nbl / 16 != 0)
		ft_p_to_hex((void *)(nbl / 16), fd, print_count);
	if (nbl % 16 > 9)
		c = ((nbl % 16) + 87);
	else
		c = ((nbl % 16) + '0');
	write(fd, &c, 1);
	(*print_count)++;
	return ;
}

/*	*** ft_putunbr_fd (42 put unsigned number file descriptor) ***
 *
 *	Takes a number and prints as an unsigned integer to file descriptor.
 */

void	ft_putunbr_fd(unsigned int n, int fd, int *print_count)
{
	unsigned long	nbl;
	char	c;

	nbl = (unsigned long)n;
	if (nbl / 10 != 0)
		ft_putunbr_fd(nbl / 10, fd, print_count);
	c = (nbl % 10 + '0');
	write(fd, &c, 1);
	(*print_count)++;
	return ;
}

Overwriting ../ft_printf/ft_printf_utils_2.c


# Testing

## ft_printf testing

In [287]:
%%file test_ft_printf.c
#include "../ft_printf/ft_printf.h"
#include <stdlib.h>
#include <stdio.h>

int main (void)
{
	char	*sample;
	char	*sample_n;

	sample = (char *)malloc(sizeof(char) * 12);
	sample_n = NULL;

	// Working
	ft_printf("*** Working ***\n");
	ft_printf(" %d\n", ft_printf("%c", '0'));
	ft_printf(" %d\n", ft_printf("Testing character: %c", '0'));
	ft_printf(" %d\n", ft_printf("Testing string: %s", "Here is a test string."));
	ft_printf(" %d\n", ft_printf("Testing decimal: %d", 12345));
	ft_printf(" %d\n", ft_printf("Testing integer: %i", 67890));
	ft_printf(" %d\n", ft_printf("Testing percent: %%"));
	ft_printf(" %d\n", ft_printf("Testing hexidecimal lower: %x", 1234));
	ft_printf(" %d\n", ft_printf("Testing hexidecimal lower: %x", -1234));
	ft_printf(" %d\n", ft_printf("Testing hexidecimal upper: %X", 1234));
	ft_printf(" %d\n", ft_printf("Testing unsigned decimal: %u", -12345));
	ft_printf(" %d\n", ft_printf("Testing ft_pointer: %p", (void *)sample));
	//	printf(" %d\n", printf("Testing    pointer: %p", (void *)sample));
	ft_printf(" %d\n", ft_printf("Testing ft_pointer: %p", (void *)sample_n));
	//	printf(" %d\n", printf("Testing    pointer: %p", (void *)sample_n));

	// Issues
	ft_printf("\n*** Issues ***\n");
	ft_printf(" %d\n", ft_printf(" NULL %s NULL ", NULL));
	printf(" %d\n", printf(" NULL %s NULL ", NULL));

	free(sample);
	return (0);
}

Overwriting test_ft_printf.c


In [288]:
%%bash
cc test_ft_printf.c -g -L../ft_printf -lftprintf
./a.out

test_ft_printf.c:33:43: warning: format specifies type 'char *' but the argument has type 'void *' [-Wformat]
        printf(" %d\n", printf(" NULL %s NULL ", NULL));
                                      ~~         ^~~~
/usr/lib/llvm-12/lib/clang/12.0.1/include/stddef.h:89:16: note: expanded from macro 'NULL'
#  define NULL ((void*)0)
               ^~~~~~~~~~
1 warning generated.


*** Working ***
0 1
Testing character: 0 20
Testing string: Here is a test string. 38
Testing decimal: 12345 22
Testing integer: 67890 22
Testing percent: % 18
Testing hexidecimal lower: 4d2 30
Testing hexidecimal lower: fffffb2e 35
Testing hexidecimal upper: 4D2 30
Testing unsigned decimal: 4294954951 36
Testing ft_pointer: 0x1cd42a0 29
Testing ft_pointer: (nil) 25

*** Issues ***
 NULL  NULL  12
 NULL (null) NULL  18


## printf testing

In [198]:
%%file test_printf.c
#include <stdio.h>
#include <stdlib.h>

int main (void)
{
	char *sample;

	sample = (char *)malloc(sizeof(char) * 12);

	printf(" %d\n", printf("%c", '0'));
	printf(" %d\n", printf("Testing character: %c", '0'));
	printf(" %d\n", printf("Testing string: %s", "Here is a test string."));
	printf(" %d\n", printf("Testing decimal: %d", 12345));
	printf(" %d\n", printf("Testing integer: %i", 67890));
	printf(" %d\n", printf("Testing percent: %%"));
	printf(" %d\n", printf("Testing hexidecimal lower: %x", 1234));
	printf(" %d\n", printf("Testing hexidecimal lower: %x", -1234));
	printf(" %d\n", printf("Testing hexidecimal upper: %X", 1234));
	printf(" %d\n", printf("Testing pointer: %p", (void *)sample));
	printf(" %d\n", printf("Testing unsigned decimal: %u", -12345));
	printf(" %d\n", printf(" NULL %s NULL ", NULL));

	free(sample);
	return (0);
}


Overwriting test_printf.c


In [199]:
%%bash
gcc test_printf.c -g
./a.out

test_printf.c: In function ‘main’:
test_printf.c:21:33: warning: format ‘%s’ expects argument of type ‘char *’, but argument 2 has type ‘void *’ [-Wformat=]
   21 |  printf(" %d\n", printf(" NULL %s NULL ", NULL));
      |                                ~^
      |                                 |
      |                                 char *
      |                                %p
test_printf.c:21:2: warning: ‘%s’ directive argument is null [-Wformat-overflow=]
   21 |  printf(" %d\n", printf(" NULL %s NULL ", NULL));
      |  ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


0 1
Testing character: 0 20
Testing string: Here is a test string. 38
Testing decimal: 12345 22
Testing integer: 67890 22
Testing percent: % 18
Testing hexidecimal lower: 4d2 30
Testing hexidecimal lower: fffffb2e 35
Testing hexidecimal upper: 4D2 30
Testing pointer: 0x5644dfd6f2a0 31
Testing unsigned decimal: 4294954951 36
 NULL (null) NULL  18


# Scratch

In [155]:
%%file glibc_printf.c

/* Copyright (C) 1991-2022 Free Software Foundation, Inc.
   This file is part of the GNU C Library.
   The GNU C Library is free software; you can redistribute it and/or
   modify it under the terms of the GNU Lesser General Public
   License as published by the Free Software Foundation; either
   version 2.1 of the License, or (at your option) any later version.
   The GNU C Library is distributed in the hope that it will be useful,
   but WITHOUT ANY WARRANTY; without even the implied warranty of
   MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU
   Lesser General Public License for more details.
   You should have received a copy of the GNU Lesser General Public
   License along with the GNU C Library; if not, see
   <https://www.gnu.org/licenses/>.  */
#include <libioP.h>
#include <stdarg.h>
#include <stdio.h>
#undef printf
/* Write formatted output to stdout from the format string FORMAT.  */
/* VARARGS1 */
int
__printf (const char *format, ...)
{
  va_list arg;
  int done;
  va_start (arg, format);
  done = __vfprintf_internal (stdout, format, arg, 0);
  va_end (arg);
  return done;
}
#undef _IO_printf
ldbl_strong_alias (__printf, printf);
ldbl_strong_alias (__printf, _IO_printf);

Overwriting glibc_printf.c
